## Doing
 - [ ] 7 day rolling averages
## To Do
 - [ ] pull data from JHU
 - [ ] pull data directly from usafacts 
 - [ ] Allow run in COLAB and save to Google Drive

## Done
 - [x] ensure proper data types in df_combined_tidy
 - [x] make tidy data one row per observation (observations are by date)
 - [x] rename variable column 'Date' in df_combined_tidy


In [1]:
import pandas as pd
import urllib3 as urllib
import json
import math

import IPython.display
import altair as alt

pd.options.display.max_columns = None

In [2]:
usafacts_confirmed_df= pd.read_csv('County-Data/covid_confirmed_usafacts.csv')
jhu_confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
    
usafacts_population_df= pd.read_csv('County-Data/covid_county_population_usafacts.csv')

In [3]:
usafacts_population_df['State_County_FIPS'] = usafacts_population_df['State'] +  usafacts_population_df['County Name'] +  usafacts_population_df['countyFIPS'].astype('str')
usafacts_confirmed_df['State_County_FIPS'] = usafacts_confirmed_df['State'] +  usafacts_confirmed_df['County Name'] +  usafacts_confirmed_df['countyFIPS'].astype('str')

In [4]:
df_confirmed = usafacts_confirmed_df[usafacts_confirmed_df['countyFIPS'] > 2].copy()
df_population = usafacts_population_df[usafacts_population_df['countyFIPS'] > 2].copy()

df_population.set_index('countyFIPS', inplace=True)
df_confirmed.set_index('countyFIPS', inplace=True)

df_confirmed_combined= pd.concat ([df_confirmed, df_population['population']], axis=1).copy()
df_confirmed_combined.reset_index(inplace=True)
# REMOVE "Counties" where population is ZERO
df_cases_clean = df_confirmed_combined[df_confirmed_combined.population != 0]
df_cases_clean.head(5)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,State_County_FIPS,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36,37,37,39,40,42,42,42,45,48,53,53,58,61,67,68,74,84,91,93,103,103,110,110,120,127,136,147,149,155,159,173,189,192,205,212,216,220,233,238,239,241,248,259,265,272,282,295,312,323,331,357,368,373,375,400,411,431,434,442,453,469,479,488,498,503,527,537,553,561,568,591,615,618,644,651,661,670,684,706,728,746,756,780,789,827,842,857,865,886,905,921,932,942,965,974,974,1002,1015,1030,1052,1066,1073,1073,1096,1113,1134,1215,1215,1215,1241,1250,1252,1262,1273,1274,1291,1293,1293,1293,1322,1324,1351,1355,1366,1377,1389,1400,1438,1442,1452,1452,1466,1475,1492,1498,1504,1508,1522,1544,1551,1565,1576,1585,1601,1619,1624,1664,1673,1690,1691,1714,1715,1715,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1

In [5]:
# date_cols = list(df_combined_clean.columns[3:-2])
id_cols = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population']

df_cases_tidy = df_cases_clean.melt(id_vars =id_cols, value_vars = df_cases_clean.columns[4:-2], var_name='Date', value_name='Cases')

df_cases_tidy['Date'] = df_cases_tidy['Date'].astype(str)
df_cases_tidy['Date'] = pd.to_datetime(df_cases_tidy['Date'], format='%m/%d/%y')

display(df_cases_tidy.dtypes)
df_cases_tidy

County Name            object
State                  object
countyFIPS              int64
stateFIPS               int64
population              int64
Date           datetime64[ns]
Cases                   int64
dtype: object

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases
0,Autauga County,AL,1001,1,55869,2020-01-22,0
1,Baldwin County,AL,1003,1,223234,2020-01-22,0
2,Barbour County,AL,1005,1,24686,2020-01-22,0
3,Bibb County,AL,1007,1,22394,2020-01-22,0
4,Blount County,AL,1009,1,57826,2020-01-22,0
...,...,...,...,...,...,...,...
1159393,Sweetwater County,WY,56037,56,42343,2021-01-24,3411
1159394,Teton County,WY,56039,56,23464,2021-01-24,2931
1159395,Uinta County,WY,56041,56,20226,2021-01-24,1900
1159396,Washakie County,WY,56043,56,7805,2021-01-24,858


In [6]:
#Save to CSV file
# df_cases_tidy.to_csv('County-Data/usafacts_cases_tidy.csv')

In [7]:
df_cases_tidy.sort_values(by=['countyFIPS', 'Date'], inplace=True)

# Step 2 - Get Rolling Averages

In [14]:
%%time
first_time = True
pd.set_option('mode.chained_assignment', None)
state = 'FL'
mydf = df_cases_tidy[(df_cases_tidy['State'] == state)]
#for countyFIPS in mydf['countyFIPS'].unique():
for countyFIPS in df_cases_tidy['countyFIPS'].unique():

    tmp_df = df_cases_tidy[df_cases_tidy['countyFIPS'] == countyFIPS]
#     tmp_df['7day'] = tmp_df.rolling(7).mean()['Cases']
    tmp_df['New Cases'] = tmp_df['Cases'] - tmp_df['Cases'].shift(1)
    tmp_df['7day'] = tmp_df['New Cases'].rolling(7).mean().round(1)
    tmp_df['prev_7day'] = tmp_df['7day'].shift(28)

    if first_time:
        df_cases_7day_tidy = tmp_df.copy()
        first_time = False
        print('First')
    else:
        df_cases_7day_tidy = df_cases_7day_tidy.append(tmp_df)

pd.set_option('mode.chained_assignment', 'warn')

First
Wall time: 2min 46s


In [15]:
print(len(df_cases_7day_tidy))
len(df_cases_tidy)

1159398


1159398

In [16]:
df_cases_7day_tidy.tail(17)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,New Cases,7day,prev_7day
1109125,Weston County,WY,56045,56,6927,2021-01-08,485,2.00,1.30,1.30
1112267,Weston County,WY,56045,56,6927,2021-01-09,485,0.00,1.10,1.30
1115409,Weston County,WY,56045,56,6927,2021-01-10,485,0.00,1.10,1.10
1118551,Weston County,WY,56045,56,6927,2021-01-11,485,0.00,0.90,1.00
1121693,Weston County,WY,56045,56,6927,2021-01-12,601,116.00,17.40,1.60
1124835,Weston County,WY,56045,56,6927,2021-01-13,601,0.00,17.40,1.30
1127977,Weston County,WY,56045,56,6927,2021-01-14,602,1.00,17.00,1.40
1131119,Weston County,WY,56045,56,6927,2021-01-15,602,0.00,16.70,2.10
1134261,Weston County,WY,56045,56,6927,2021-01-16,602,0.00,16.70,2.10
1137403,Weston County,WY,56045,56,6927,2021-01-17,602,0.00,16.70,2.60


In [17]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)


pd.set_option('mode.chained_assignment', None)
a=df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()]

a['7day/1000'] = a['7day'] / a['population'] * 1000
a['prev_7day/1000'] = a['prev_7day'] / a['population'] * 1000
a['diff prev7'] =  a['7day']  - a['prev_7day']
a['diff prev7/1000'] = a['diff prev7'] / a['population'] * 1000

sort_by='diff prev7/1000'
a.sort_values(by=sort_by, ascending=False, inplace=True)
pd.set_option('mode.chained_assignment', 'warn')
a.head(10)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,New Cases,7day,prev_7day,7day/1000,prev_7day/1000,diff prev7,diff prev7/1000
1158526,Forest County,PA,42053,42,7247,2021-01-24,1247,1.00,94.60,11.10,13.05,1.53,83.50,11.52
1156557,San Miguel County,CO,8113,8,8179,2021-01-24,597,3.00,18.70,2.40,2.29,0.29,16.30,1.99
1159289,Randolph County,WV,54083,54,28695,2021-01-24,2095,10.00,59.70,11.00,2.08,0.38,48.70,1.70
1156356,Graham County,AZ,4009,4,38837,2021-01-24,4816,37.00,65.00,0.00,1.67,0.00,65.00,1.67
1156317,Tallapoosa County,AL,1123,1,40367,2021-01-24,3149,6.00,76.10,18.00,1.89,0.45,58.10,1.44
1159044,Juab County,UT,49023,49,12017,2021-01-24,1050,0.00,16.60,0.00,1.38,0.00,16.60,1.38
1157724,Tallahatchie County,MS,28135,28,13809,2021-01-24,1572,10.00,24.40,6.90,1.77,0.50,17.50,1.27
1159285,Pocahontas County,WV,54075,54,8247,2021-01-24,545,0.00,13.40,3.10,1.62,0.38,10.30,1.25
1156556,San Juan County,CO,8111,8,728,2021-01-24,40,0.00,0.90,0.00,1.24,0.00,0.90,1.24
1156337,Kusilvak Census Area,AK,2158,2,8314,2021-01-24,1082,9.00,12.70,2.70,1.53,0.32,10.00,1.20


In [18]:
for countyFIPS in a['countyFIPS'].head(10):
    fips_df = df_cases_7day_tidy[df_cases_7day_tidy['countyFIPS'] == countyFIPS]
    field = '7day'
    field_title = 'New Cases 7 Day Average'
    chart_title = a.iloc[0]['State'] + ': ' + a.iloc[0]['County Name']
    chart1 = alt.Chart(fips_df).mark_bar().encode(
                         x=alt.X('yearmonthdate(Date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                         y=alt.Y(field, title=field_title),
                         ).properties( width=400, height=400, title=chart_title)
    display(chart1)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [19]:
a.iloc[0]['State'] + ': '' + a.iloc[0]['County Name'] 
# a.columns

SyntaxError: invalid syntax (<ipython-input-19-c00617ad26ce>, line 1)

# PLAY AREA

In [ ]:
fips_df

In [ ]:
field = 'New Cases'
chart_title = 'Cas'
chart1 = alt.Chart(fips_df).mark_bar().encode(
                     x=alt.X('yearmonthdate(Date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y=alt.Y(field, title=field_title),
                     ).properties( width=400, height=400, title=chart_title)
chart1

In [ ]:
chart1